# Permutation algorithm, using combinations of two events

In [1]:
import pandas as pd
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt
import os
import numpy
import itertools
from itertools import tee, combinations, permutations
from progressbar import ProgressBar
import datetime
from sklearn.metrics import mean_squared_error, accuracy_score
import time
import calendar
from statistics import median

In [2]:
start_time = time.time()

In [3]:
os.chdir("../")

# Training

In [38]:
data_train = pd.read_csv('./data/2017-train.csv', error_bad_lines=False)
data_test = pd.read_csv('./data/2017-test.csv', error_bad_lines=False)

In [44]:
data_train['event time:timestamp'] = pd.to_datetime(data_train['event time:timestamp'])

In [39]:
data_train = data_train.sort_values(by=['case concept:name', 'event time:timestamp'])

In [45]:
data_train['day_of_week'] = data_train['event time:timestamp'].dt.dayofweek

In [42]:
### Addition
data_train['case LoanGoal'] = data_train['case LoanGoal'].apply(lambda x: 'Other - see explanation' if x == 'Other, see explanation' else x)

In [47]:
data_train.to_csv("fixed.csv")

In [48]:
#file = open('fixed.csv', 'r')
log = dict()

with open('fixed.csv', 'r') as file:
    next(file)
    for line in file:
        line = line.strip()
        if len(line) == 0:
            continue
        parts = line.split(',')
        ### Numbers changed here
        caseid = parts[4]
        
        task = parts[8]
        timestamp = parts[12]
        day = parts[13]


        if caseid not in log:
            log[caseid] = [[],[],[]]

        log[caseid][0].append(task)
        log[caseid][1].append(timestamp)
        log[caseid][2].append(day)

file.close()
#os.remove('fixed.csv')

In [49]:
log

{'Application_1000086665': [['A_Create Application',
   'A_Submitted',
   'W_Handle leads',
   'W_Handle leads',
   'W_Complete application',
   'A_Concept',
   'W_Complete application',
   'W_Complete application',
   'A_Accepted',
   'O_Create Offer',
   'O_Created',
   'O_Sent (mail and online)',
   'W_Complete application',
   'W_Call after offers',
   'W_Call after offers',
   'A_Complete',
   'W_Call after offers',
   'A_Cancelled',
   'O_Cancelled',
   'W_Call after offers',
   'W_Call after offers',
   'W_Call after offers'],
  ['2016-03-08 17:57:21.673',
   '2016-03-08 17:57:21.734',
   '2016-03-08 17:57:21.963',
   '2016-03-08 17:58:28.286',
   '2016-03-08 17:58:28.293',
   '2016-03-08 17:58:28.299',
   '2016-04-08 15:39:29.557',
   '2016-04-08 15:50:12.281',
   '2016-05-08 15:57:07.419',
   '2016-05-08 15:59:57.320',
   '2016-05-08 15:59:58.162',
   '2016-05-08 16:01:23.264',
   '2016-05-08 16:01:23.278',
   '2016-05-08 16:01:23.284',
   '2016-05-08 16:01:23.286',
   '2016-0

# Event Prediction

In [50]:
def pairwise(iterable):
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)

In [51]:
event_names = list(data_train['event concept:name'].unique())
event_names.append('New Event')
combs = []

for p in itertools.product(event_names, repeat=2):
    combs.append(p)

In [53]:
len(combs)

729

In [54]:
#del log['case concept:name']

pbar = ProgressBar()

for i in pbar(log.keys()):
    ID = []
    stamps = []
    for pairID, stamp in zip(pairwise(log[i][0]), pairwise(log[i][1])):
        ID.append(pairID)
        stamps.append(stamp)
            
    log[i].append(ID)
    log[i].append(stamps)

100% |########################################################################|


In [55]:
pbar = ProgressBar()
count=0

for i in pbar(log.keys()):
    count=0
    for perm in log[i][3]:
        index = log[i][3].index(perm)
        if index == 0:
            log[i][3].insert(0, ('New Event', log[i][3][index][0]))
            
    for count in range(len(log[i][4])):
        stamp = log[i][4][count]
        if count == 0:
            log[i][4].insert(0, (stamp[0], stamp[0]))
            
        count += 1

100% |########################################################################|


In [63]:
def timeDiff(tupl):
    
    datetimeFormat = '%Y-%m-%d %H:%M:%S.%f'
    diff = datetime.datetime.strptime(tupl[0], datetimeFormat)\
    - datetime.datetime.strptime(tupl[1], datetimeFormat)
    
    return abs(diff.days)

In [57]:
def listCount(lst: list): 
    cases = list(data_train['event concept:name'].unique())  # list of all unique event names
    cases.append('New Event')
    best = 0
    for x in cases:
        current = lst.count(x)
        if current >= best:
            best = current
            str_best = x
    return str_best

In [58]:
pbar = ProgressBar() 

comb_times = {}

for comb in pbar(combs):
    day_list = [[], [], [], [], [], [], []]
    
    for case in log.keys():
        if comb in log[case][3]:     
            index = log[case][3].index(comb)
            day = int(log[case][2][index])
            
            if index < (len(log[case][3]) - 1):
                nxt_event = log[case][3][index+1][1]  # we need the second item of the tuple, bc item 1 is repeated
                day_list[day].append(nxt_event)

            elif index == (len(log[case][3]) - 1):
                nxt_event = 'New Event'
                day_list[day].append(nxt_event)

        else:
            pass
        
    comb_times[comb] = day_list
    
    for i in range(len(day_list)):
        comb_times[comb][i] = listCount(comb_times[comb][i])
    

100% |########################################################################|


In [59]:
comb_times

{('A_Create Application', 'A_Create Application'): ['New Event',
  'New Event',
  'New Event',
  'New Event',
  'New Event',
  'New Event',
  'New Event'],
 ('A_Create Application', 'A_Submitted'): ['W_Handle leads',
  'W_Handle leads',
  'W_Handle leads',
  'W_Handle leads',
  'W_Handle leads',
  'W_Handle leads',
  'W_Handle leads'],
 ('A_Create Application', 'W_Handle leads'): ['New Event',
  'New Event',
  'New Event',
  'New Event',
  'New Event',
  'New Event',
  'New Event'],
 ('A_Create Application', 'W_Complete application'): ['W_Complete application',
  'W_Complete application',
  'W_Complete application',
  'W_Complete application',
  'W_Complete application',
  'W_Complete application',
  'W_Complete application'],
 ('A_Create Application', 'A_Concept'): ['W_Complete application',
  'W_Complete application',
  'W_Complete application',
  'W_Complete application',
  'W_Complete application',
  'W_Complete application',
  'W_Complete application'],
 ('A_Create Application', '

# Adding Predictions

In [64]:
pbar = ProgressBar()

for i in pbar(log.keys()):
# Add the real time differences
    real_diff = []
    for t in log[i][4]:
        real_diff.append(timeDiff(t))
    log[i].extend([real_diff])

100% |########################################################################|


In [65]:
"""Adding predictions based on the combination with respect to the week. """

for i in log.keys():
    current = log[i][3]
    prediction = []
    
    for perm in current:
        index = current.index(perm)
        day = int(log[i][2][index])
        current_prediction = comb_times[perm][day]
        
        if current_prediction != 0:
            prediction.append(current_prediction)
        else:
            merged_list = list(itertools.chain.from_iterable(comb_times[perm]))
            pred = listCount(merged_list)
            prediction.append(pred)

    log[i].extend([prediction])
    

    current_real = []
    
    for x in log[i][0]:
        if log[i][0].index(x) == 0:
            current_real.append('New Event')
        else:
            current_real.append(x)
    log[i].extend([current_real]) 


In [66]:
"""Storing all time differences for every combination."""

pbar = ProgressBar()

times = {}
for comb in pbar(combs):
    for case in log.keys():
        if comb in log[case][3]:
            count = log[case][3].index(comb)
            diff = timeDiff(log[case][4][count])
            if comb not in times:
                times[comb] = []
                times[comb].append(diff)
            else:
                times[comb].append(diff)
        else:
            pass
    if comb in times.keys():
        times[comb] = int(np.ceil(np.mean(times[comb])))

100% |########################################################################|


In [67]:
"""Prediction for time difference, we check whether event is last and then predict 0 for it!"""
for i in log.keys():
    time_pred = []
    for ev, pred, day in zip(log[i][0], log[i][6], log[i][2]):
        last = len(log[i][0]) - 1
        
        if log[i][0].index(ev) == last:
            time_pred.append(0)
        elif pred == 'New Event':
            time_pred.append(0)
        elif (ev, pred) in times:
            if int(day) + times[(ev,pred)] % 7 == 5:
                time_pred.append(times[(ev,pred)] + 2)
            elif int(day) + times[(ev,pred)] % 7 == 6:
                time_pred.append(times[(ev,pred)] + 1)
            else:
                time_pred.append(times[(ev,pred)])

    log[i].extend([time_pred])

In [68]:
case_names = [] 
event_names = []
timestamp = []
p_event = []
current_real = []

real_diff = []
pred_diff = []

for i in log.keys():
    for x in range(len(log[i][0])):
        case_names.append(i)
        event_names.append(log[i][0][x])
        timestamp.append(log[i][1][x])
        p_event.append(log[i][6][x])
        current_real.append(log[i][7][x])
        
        real_diff.append(log[i][5][x])
        pred_diff.append(log[i][8][x])

real_diff.append(0)
        
frame_dict = {'Case_ID': case_names, 'Event_Name': event_names,
              'TimeStamp': timestamp, 'Current_Event': current_real, 'Predicted_Event': p_event,
            'Real_Diff': real_diff[1:], 'Predicted_Diff': pred_diff}
predicted_df = pd.DataFrame.from_dict(frame_dict)


In [71]:
predicted_df

,Case_ID,Event_Name,TimeStamp,Current_Event,Predicted_Event,Real_Diff,Predicted_Diff
0,Application_1000086665,A_Create Application,2016-03-08 17:57:21.673,New Event,A_Submitted,1,1
1,Application_1000086665,A_Submitted,2016-03-08 17:57:21.734,A_Submitted,W_Handle leads,1,1
2,Application_1000086665,W_Handle leads,2016-03-08 17:57:21.963,W_Handle leads,W_Handle leads,1,2
3,Application_1000086665,W_Handle leads,2016-03-08 17:58:28.286,W_Handle leads,W_Complete application,1,1
4,Application_1000086665,W_Complete application,2016-03-08 17:58:28.293,W_Complete application,A_Concept,1,1
5,Application_1000086665,A_Concept,2016-03-08 17:58:28.299,A_Concept,W_Complete application,31,20
6,Application_1000086665,W_Complete application,2016-04-08 15:39:29.557,W_Complete application,W_Complete application,1,3
7,Application_1000086665,W_Complete application,2016-04-08 15:50:12.281,W_Complete application,A_Concept,31,3
8,Application_1000086665,A_Accepted,2016-05-08 15:57:07.419,A_Accepted,O_Create Offer,1,2
9,Application_1000086665,O_Create Offer,2016-05-08 15:59:57.320,O_Create Offer,O_Created,1,1


# Evaluation

In [69]:
event_real = np.array(predicted_df['Current_Event'])
event_pred = np.array(predicted_df['Predicted_Event'])
event_real = event_real[1:]
event_pred = event_pred[:-1]

acc = accuracy_score(event_real, event_pred)
print('Accuracy for event prediction TRAIN SET: {}%'.format(round(acc,2)*100))

Accuracy for event prediction TRAIN SET: 59.0%


In [70]:
time_real = np.array(predicted_df['Real_Diff'])
time_pred = np.array(predicted_df['Predicted_Diff'])

time_pred = time_pred
time_real = time_real

rms = np.sqrt(mean_squared_error(time_real, time_pred))
print('Root mean squared error for time difference prediction TRAIN SET: {}'.format(round(rms,2)))

Root mean squared error for time difference prediction TRAIN SET: 25.93


# Test

In [23]:
data_test['event time:timestamp'] = pd.to_datetime(data_test['event time:timestamp'])

data_test['event time:timestamp'] = pd.to_datetime(data_test['event time:timestamp'])

data_test = data_test.sort_values(by=['case concept:name', 'event time:timestamp'])

data_test['day_of_week'] = data_test['event time:timestamp'].dt.dayofweek

data_test.to_csv("fixed_test.csv")

In [24]:
file_test = open('fixed_test.csv', 'r')
t_log = dict()

for line in file_test:
    line = line.strip()
    if len(line) == 0:
        continue
    parts = line.split(',')
    
    caseid = parts[2]
    
    task = parts[3]
    timestamp = parts[5]
    day = parts[6]
    
    if caseid not in t_log:
        t_log[caseid] = [[],[],[]]

    t_log[caseid][0].append(task)
    t_log[caseid][1].append(timestamp)
    t_log[caseid][2].append(day)
    
file.close()
os.remove('fixed_test.csv')

del t_log['case concept:name']

# Test prediction

In [25]:
pbar = ProgressBar()

for i in pbar(t_log.keys()):
    ID = []
    stamps = []
    for pairID, stamp in zip(pairwise(t_log[i][0]), pairwise(t_log[i][1])):
        ID.append(pairID)
        stamps.append(stamp)

    t_log[i].append(ID)
    t_log[i].append(stamps)

pbar = ProgressBar()
count = 0

for i in pbar(t_log.keys()):
    count = 0
    for perm in t_log[i][3]:
        index = t_log[i][3].index(perm)
        if index == 0:
            t_log[i][3].insert(0, ('New Event', t_log[i][3][index][0]))

    for count in range(len(t_log[i][4])):
        stamp = t_log[i][4][count]
        if count == 0:
            t_log[i][4].insert(0, (stamp[0], stamp[0]))

        count += 1

pbar = ProgressBar()

for i in pbar(t_log.keys()):
    # Add the real time differences
    real_diff = []
    for t in t_log[i][4]:
        real_diff.append(timeDiff(t))
    t_log[i].extend([real_diff])

"""Adding predictions based on the combination with respect to the week. """

for i in t_log.keys():
    current = t_log[i][3]
    prediction = []

    for perm in current:
        index = current.index(perm)
        day = int(t_log[i][2][index])
        current_prediction = comb_times[perm][day]

        if current_prediction != 0:
            prediction.append(current_prediction)
        else:
            merged_list = list(itertools.chain.from_iterable(comb_times[perm]))
            pred = listCount(merged_list)
            prediction.append(pred)

    t_log[i].extend([prediction])

    current_real = []

    for x in t_log[i][0]:
        if t_log[i][0].index(x) == 0:
            current_real.append('New Event')
        else:
            current_real.append(x)
    t_log[i].extend([current_real])

"""Prediction for time difference, we check whether event is last and then predict 0 for it!"""
for i in t_log.keys():
    time_pred = []
    for ev, pred, day in zip(t_log[i][0], t_log[i][6], t_log[i][2]):
        last = len(t_log[i][0]) - 1

        if t_log[i][0].index(ev) == last:
            time_pred.append(0)
        elif pred == 'New Event':
            time_pred.append(0)
        elif (ev, pred) in times:
            if int(day) + times[(ev, pred)] % 7 == 5:
                time_pred.append(times[(ev, pred)] + 2)
            elif int(day) + times[(ev, pred)] % 7 == 6:
                time_pred.append(times[(ev, pred)] + 1)
            else:
                time_pred.append(times[(ev, pred)])

    t_log[i].extend([time_pred])

case_names = []
event_names = []
timestamp = []
p_event = []
current_real = []

real_diff = []
pred_diff = []

for i in t_log.keys():
    for x in range(len(t_log[i][0])):
        case_names.append(i)
        event_names.append(t_log[i][0][x])
        timestamp.append(t_log[i][1][x])
        p_event.append(t_log[i][6][x])
        current_real.append(t_log[i][7][x])

        real_diff.append(t_log[i][5][x])
        pred_diff.append(t_log[i][8][x])

real_diff.append(0)

frame_dict = {'Case_ID': case_names, 'Event_Name': event_names,
              'TimeStamp': timestamp, 'Current_Event': current_real, 'Predicted_Event': p_event,
              'Real_Diff': real_diff[1:], 'Predicted_Diff': pred_diff}
predicted_df = pd.DataFrame.from_dict(frame_dict)

event_real = np.array(predicted_df['Current_Event'])
event_pred = np.array(predicted_df['Predicted_Event'])
event_real = event_real[1:]
event_pred = event_pred[:-1]

acc = accuracy_score(event_real, event_pred)
print('Accuracy for event prediction TEST SET: {}%'.format(round(acc, 2) * 100))

time_real = np.array(predicted_df['Real_Diff'])
time_pred = np.array(predicted_df['Predicted_Diff'])

time_pred = time_pred
time_real = time_real

rms = np.sqrt(mean_squared_error(time_real, time_pred))
print('Root mean squared error for time difference prediction TEST SET: {}'.format(round(rms, 2)))

100% |########################################################################|
100% |########################################################################|
100% |########################################################################|


Accuracy for event prediction TEST SET: 70.0%
Root mean squared error for time difference prediction TEST SET: 172.17


In [26]:
print("--- %s seconds ---" % (time.time() - start_time))

--- 123.22996497154236 seconds ---
